In [1]:
import numpy as np
import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt
import os.path
from bs4 import BeautifulSoup
import requests

In [2]:
first_record = 0
last_record = 10000
record_segment_file_name = '../../../2_data/' + 'record_segment_good_length_' + str(first_record) + '_' + str(last_record - 1) +'.csv'

In [3]:
record_segment_file_name

'../../../2_data/record_segment_good_length_0_9999.csv'

In [4]:
record_path_local = '../../../2_data/record_path_mimic3_matched.csv'
record_root_url = 'https://physionet.org/content/mimic3wdb/1.0/matched'
record_path_url = record_root_url + '/RECORDS-waveforms'

signal_extract_name = 'PLETH'
sampling_frequency = 125 # Hz
signal_extract_minimum_length = 5 # the minimum length of the signal segment, in minutes

print('record_path_local: {}'.format(record_path_local))
print('record_root_url: {}'.format(record_root_url))
print('record_path_url: {}'.format(record_path_url))

record_path_local: ../../../2_data/record_path_mimic3_matched.csv
record_root_url: https://physionet.org/content/mimic3wdb/1.0/matched
record_path_url: https://physionet.org/content/mimic3wdb/1.0/matched/RECORDS-waveforms


In [5]:
def get_url_content(url, tag=None):
    
    '''
    This function scrapes a list of useful information from a given PhysioNet URL.
    If the URL address points to an HTML document, the information to be extracted is define by a tag.
    I found this address (https://hackersandslackers.com/scraping-urls-with-beautifulsoup/) usefull
      for details on how to scrape a web page
    '''
    
    headers = {
        'Access-Control-Allow-Origin': '*',
        'Access-Control-Allow-Methods': 'GET',
        'Access-Control-Allow-Headers': 'Content-Type',
        'Access-Control-Max-Age': '3600',
        'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:52.0) Gecko/20100101 Firefox/52.0'
    }    
    
    req = requests.get(url, headers)
    soup = BeautifulSoup(req.content, 'html.parser')
    if '<!DOCTYPE html>' in str(soup):
        content = getattr(soup, str(tag)).getText()
        #content = soup.pre.getText()
    else:
        content = soup.getText()
    return content.splitlines()

In [6]:
def get_signal_name(url_content):
    return [x.split(' ')[-1] for x in url_content[1:]]

In [7]:
tag = 'pre'
record_path_list = get_url_content(record_path_url, tag)
print(len(record_path_list))
for i in range(5):
    print('record_path: {}'.format(record_path_list[i]))

22317
record_path: p00/p000020/p000020-2183-04-28-17-47
record_path: p00/p000030/p000030-2172-10-16-12-22
record_path: p00/p000033/p000033-2116-12-24-12-35
record_path: p00/p000033/p000033-2116-12-25-13-11
record_path: p00/p000052/p000052-2191-01-10-02-21


In [ ]:
tag = 'pre'
#last_record = len(record_path_list) + 1
for process_record_index in range(first_record, last_record):
        
    #print(process_record_index)
    record_path = record_path_list[process_record_index]
    record_path_short = record_path.split('/')[2]
    print('processing record #: {}, record_path_short: {}'.format(process_record_index, record_path_short))
    
    directory = str(record_path.split("/")[0]) + '/' + str(record_path.split("/")[1])
    #print('directory: {}'.format(directory))
            
    master_waveform_header = record_root_url + '/' + record_path + '.hea'
    #print('master_waveform_header: {}'.format(master_waveform_header))
    
    target_url = master_waveform_header
    master_waveform_content = get_url_content(target_url, tag)
    #print('master_waveform_content: {}'.format(master_waveform_content))
    #print('       ...........\n')
    
    master_waveform_content_first_line = master_waveform_content[0].split(' ')
    record_length = int(master_waveform_content_first_line[3]) / sampling_frequency
    #print('record_length (s): {}'.format(record_length))
    #print('       ...........\n')
        
    waveform_layout_header = master_waveform_content[1].split(' ')[0]
    #print('1. waveform_layout_header: {}\n'.format(waveform_layout_header))
    
    waveform_layout_header = waveform_layout_header + '.hea'
    #print('2. waveform_layout_header: {}\n'.format(waveform_layout_header))
    
    waveform_layout_header = record_root_url + '/' + directory + '/' + waveform_layout_header
    #print('record_root_url: {}'.format(record_root_url))
    #print('directory: {}'.format(directory))
    #print('3. waveform_layout_header: {}\n'.format(waveform_layout_header))
    #print('waveform_layout_header: {}\n'.format(waveform_layout_header))
    
    target_url = waveform_layout_header
    waveform_layout_content = get_url_content(target_url, tag)
    #print('waveform_layout_content: {}'.format(waveform_layout_content))
        
    record_signal_names = get_signal_name(waveform_layout_content)
    #print('record_signal_names: {}'.format(record_signal_names))
    
    if signal_extract_name in record_signal_names and record_length > 60 * signal_extract_minimum_length: # and len(record_segments_path_list):      
        
        # Open the record_segment_file_name file with access mode 'a'
        record_segment_file_object = open(record_segment_file_name, 'a')

        # get the waveform record name
        waveform_record_id = [x.split(' ')[0].split('_') for x in master_waveform_content if '3' == x[0]][0][0]
        #print('waveform_record_id: {}\n'.format(waveform_record_id))
        
        # get the waveform_record_state
        waveform_record_state = [(waveform_record_id + '_' in x) and not (waveform_record_id + '_layout' in x) 
                for x in master_waveform_content]
        #print('waveform_record_state: {}\n'.format(waveform_record_state))

        # get the waveform_record_segment
        waveform_record_segment = [master_waveform_content[x].split(' ') for x, y in enumerate(waveform_record_state) if y]
        #print('waveform_record_segment: {}\n'.format(waveform_record_segment))
      
        for segment_index in range(len(waveform_record_segment)):
            # number of samples: int(waveform_record_segment[segment_index][1])
            # sampling frequency: int(master_waveform_content_first_line[2])
            # length (in seconds) of the record segment: number of samples / sampling frequency
            
            # https://physionet.org/content/mimic3wdb/1.0/matched/p00/p000107/3168852_0001.hea
            
            segment_header = record_root_url + '/' + directory + '/' + waveform_record_segment[segment_index][0] + '.hea'
            #print(segment_header)
            
            segment_header_content = get_url_content(segment_header, tag)
            #print('segment_header_content: {}\n'.format(segment_header_content))
            
            segment_signal_names = get_signal_name(segment_header_content)
            #print('segment_signal_names: {}\n'.format(segment_signal_names))
           
            waveform_record_segment_length = int(waveform_record_segment[segment_index][1]) / sampling_frequency
            #print('waveform_record_segment_length: {}\n'.format(waveform_record_segment_length))
            
            if signal_extract_name in segment_signal_names and waveform_record_segment_length > 60 * signal_extract_minimum_length:
                
                waveform_record_segment_good_length = waveform_record_segment[segment_index][0]
                print('  ' + str(process_record_index) + ',' + record_path + ',' + waveform_record_segment_good_length)                
                
                # Append the waveform_record_segment_good_length_root at the end of file record_segment_file_name
                record_segment_file_object.write(str(process_record_index) + ',' + record_path + ',' + waveform_record_segment_good_length + '\n')
        
        # Close the file record_segment_file_name
        record_segment_file_object.close()
        
    #index_of_last_record_processed = index_of_last_record_processed + 1
    print('  \nindex of last record processed successfully: {}'.format(process_record_index))  
    print('--------------------------------------')

    
    


processing record #: 0, record_path_short: p000020-2183-04-28-17-47
  
index of last record processed successfully: 0
--------------------------------------
processing record #: 1, record_path_short: p000030-2172-10-16-12-22
  
index of last record processed successfully: 1
--------------------------------------
processing record #: 2, record_path_short: p000033-2116-12-24-12-35
  
index of last record processed successfully: 2
--------------------------------------
processing record #: 3, record_path_short: p000033-2116-12-25-13-11
  
index of last record processed successfully: 3
--------------------------------------
processing record #: 4, record_path_short: p000052-2191-01-10-02-21
  4,p00/p000052/p000052-2191-01-10-02-21,3238451_0005
  
index of last record processed successfully: 4
--------------------------------------
processing record #: 5, record_path_short: p000052-2191-01-10-12-15
  5,p00/p000052/p000052-2191-01-10-12-15,3533390_0001
  5,p00/p000052/p000052-2191-01-10-12-1

  14,p00/p000109/p000109-2142-01-14-18-53,3407610_0004
  14,p00/p000109/p000109-2142-01-14-18-53,3407610_0006
  14,p00/p000109/p000109-2142-01-14-18-53,3407610_0009
  14,p00/p000109/p000109-2142-01-14-18-53,3407610_0013
  14,p00/p000109/p000109-2142-01-14-18-53,3407610_0015
  14,p00/p000109/p000109-2142-01-14-18-53,3407610_0017
  14,p00/p000109/p000109-2142-01-14-18-53,3407610_0019
  14,p00/p000109/p000109-2142-01-14-18-53,3407610_0020
  14,p00/p000109/p000109-2142-01-14-18-53,3407610_0023
  14,p00/p000109/p000109-2142-01-14-18-53,3407610_0025
  14,p00/p000109/p000109-2142-01-14-18-53,3407610_0028
  
index of last record processed successfully: 14
--------------------------------------
processing record #: 15, record_path_short: p000109-2142-01-14-18-57
  15,p00/p000109/p000109-2142-01-14-18-57,3943625_0004
  15,p00/p000109/p000109-2142-01-14-18-57,3943625_0006
  15,p00/p000109/p000109-2142-01-14-18-57,3943625_0009
  15,p00/p000109/p000109-2142-01-14-18-57,3943625_0013
  15,p00/p000109

  27,p00/p000124/p000124-2166-01-09-13-03,3931528_0207
  27,p00/p000124/p000124-2166-01-09-13-03,3931528_0209
  27,p00/p000124/p000124-2166-01-09-13-03,3931528_0211
  
index of last record processed successfully: 27
--------------------------------------
processing record #: 28, record_path_short: p000124-2166-01-11-23-28
  28,p00/p000124/p000124-2166-01-11-23-28,3776642_0006
  28,p00/p000124/p000124-2166-01-11-23-28,3776642_0008
  28,p00/p000124/p000124-2166-01-11-23-28,3776642_0014
  28,p00/p000124/p000124-2166-01-11-23-28,3776642_0016
  28,p00/p000124/p000124-2166-01-11-23-28,3776642_0018
  28,p00/p000124/p000124-2166-01-11-23-28,3776642_0019
  28,p00/p000124/p000124-2166-01-11-23-28,3776642_0020
  28,p00/p000124/p000124-2166-01-11-23-28,3776642_0023
  28,p00/p000124/p000124-2166-01-11-23-28,3776642_0026
  28,p00/p000124/p000124-2166-01-11-23-28,3776642_0028
  28,p00/p000124/p000124-2166-01-11-23-28,3776642_0030
  28,p00/p000124/p000124-2166-01-11-23-28,3776642_0032
  28,p00/p000124

  45,p00/p000188/p000188-2162-01-10-18-53,3285727_0011
  45,p00/p000188/p000188-2162-01-10-18-53,3285727_0013
  45,p00/p000188/p000188-2162-01-10-18-53,3285727_0015
  45,p00/p000188/p000188-2162-01-10-18-53,3285727_0018
  45,p00/p000188/p000188-2162-01-10-18-53,3285727_0020
  45,p00/p000188/p000188-2162-01-10-18-53,3285727_0023
  45,p00/p000188/p000188-2162-01-10-18-53,3285727_0025
  45,p00/p000188/p000188-2162-01-10-18-53,3285727_0027
  45,p00/p000188/p000188-2162-01-10-18-53,3285727_0029
  45,p00/p000188/p000188-2162-01-10-18-53,3285727_0032
  45,p00/p000188/p000188-2162-01-10-18-53,3285727_0035
  45,p00/p000188/p000188-2162-01-10-18-53,3285727_0037
  45,p00/p000188/p000188-2162-01-10-18-53,3285727_0040
  45,p00/p000188/p000188-2162-01-10-18-53,3285727_0042
  45,p00/p000188/p000188-2162-01-10-18-53,3285727_0045
  45,p00/p000188/p000188-2162-01-10-18-53,3285727_0048
  45,p00/p000188/p000188-2162-01-10-18-53,3285727_0049
  45,p00/p000188/p000188-2162-01-10-18-53,3285727_0053
  45,p00/p

  73,p00/p000333/p000333-2147-04-25-22-02,3092245_0032
  73,p00/p000333/p000333-2147-04-25-22-02,3092245_0039
  73,p00/p000333/p000333-2147-04-25-22-02,3092245_0040
  73,p00/p000333/p000333-2147-04-25-22-02,3092245_0041
  73,p00/p000333/p000333-2147-04-25-22-02,3092245_0043
  73,p00/p000333/p000333-2147-04-25-22-02,3092245_0044
  73,p00/p000333/p000333-2147-04-25-22-02,3092245_0045
  73,p00/p000333/p000333-2147-04-25-22-02,3092245_0046
  73,p00/p000333/p000333-2147-04-25-22-02,3092245_0047
  73,p00/p000333/p000333-2147-04-25-22-02,3092245_0050
  73,p00/p000333/p000333-2147-04-25-22-02,3092245_0051
  73,p00/p000333/p000333-2147-04-25-22-02,3092245_0054
  73,p00/p000333/p000333-2147-04-25-22-02,3092245_0055
  73,p00/p000333/p000333-2147-04-25-22-02,3092245_0056
  73,p00/p000333/p000333-2147-04-25-22-02,3092245_0060
  73,p00/p000333/p000333-2147-04-25-22-02,3092245_0061
  73,p00/p000333/p000333-2147-04-25-22-02,3092245_0062
  73,p00/p000333/p000333-2147-04-25-22-02,3092245_0063
  73,p00/p

...

  1888,p00/p007654/p007654-2171-12-24-10-18,3020774_0014
  1888,p00/p007654/p007654-2171-12-24-10-18,3020774_0015
  1888,p00/p007654/p007654-2171-12-24-10-18,3020774_0018
  1888,p00/p007654/p007654-2171-12-24-10-18,3020774_0020
  1888,p00/p007654/p007654-2171-12-24-10-18,3020774_0022
  1888,p00/p007654/p007654-2171-12-24-10-18,3020774_0024
  1888,p00/p007654/p007654-2171-12-24-10-18,3020774_0027
  1888,p00/p007654/p007654-2171-12-24-10-18,3020774_0030
  1888,p00/p007654/p007654-2171-12-24-10-18,3020774_0033
  1888,p00/p007654/p007654-2171-12-24-10-18,3020774_0036
  1888,p00/p007654/p007654-2171-12-24-10-18,3020774_0038
  
index of last record processed successfully: 1888
--------------------------------------
processing record #: 1889, record_path_short: p007655-2144-09-03-22-17
  
index of last record processed successfully: 1889
--------------------------------------
processing record #: 1890, record_path_short: p007655-2144-09-04-15-47
  
index of last record processed successfully:

  1920,p00/p007755/p007755-2120-03-13-09-14,3659274_0001
  1920,p00/p007755/p007755-2120-03-13-09-14,3659274_0004
  1920,p00/p007755/p007755-2120-03-13-09-14,3659274_0005
  
index of last record processed successfully: 1920
--------------------------------------
processing record #: 1921, record_path_short: p007758-2114-10-30-15-59
  
index of last record processed successfully: 1921
--------------------------------------
processing record #: 1922, record_path_short: p007758-2114-10-31-15-24
  
index of last record processed successfully: 1922
--------------------------------------
processing record #: 1923, record_path_short: p007760-2177-09-30-12-00
  
index of last record processed successfully: 1923
--------------------------------------
processing record #: 1924, record_path_short: p007760-2177-09-30-14-07
  
index of last record processed successfully: 1924
--------------------------------------
processing record #: 1925, record_path_short: p007782-2157-03-28-05-50
  
index of la

  1947,p00/p007809/p007809-2139-07-02-18-51,3393744_0001
  1947,p00/p007809/p007809-2139-07-02-18-51,3393744_0003
  1947,p00/p007809/p007809-2139-07-02-18-51,3393744_0005
  1947,p00/p007809/p007809-2139-07-02-18-51,3393744_0007
  1947,p00/p007809/p007809-2139-07-02-18-51,3393744_0009
  1947,p00/p007809/p007809-2139-07-02-18-51,3393744_0011
  
index of last record processed successfully: 1947
--------------------------------------
processing record #: 1948, record_path_short: p007809-2139-07-03-17-51
  1948,p00/p007809/p007809-2139-07-03-17-51,3695060_0005
  
index of last record processed successfully: 1948
--------------------------------------
processing record #: 1949, record_path_short: p007809-2139-07-04-20-53
  1949,p00/p007809/p007809-2139-07-04-20-53,3654087_0001
  1949,p00/p007809/p007809-2139-07-04-20-53,3654087_0003
  1949,p00/p007809/p007809-2139-07-04-20-53,3654087_0005
  1949,p00/p007809/p007809-2139-07-04-20-53,3654087_0007
  
index of last record processed successfully:

  
index of last record processed successfully: 1973
--------------------------------------
processing record #: 1974, record_path_short: p007965-2136-06-18-15-17
  
index of last record processed successfully: 1974
--------------------------------------
processing record #: 1975, record_path_short: p007965-2141-11-27-20-18
  1975,p00/p007965/p007965-2141-11-27-20-18,3376765_0004
  
index of last record processed successfully: 1975
--------------------------------------
processing record #: 1976, record_path_short: p007965-2141-11-28-14-41
  1976,p00/p007965/p007965-2141-11-28-14-41,3476791_0007
  1976,p00/p007965/p007965-2141-11-28-14-41,3476791_0008
  1976,p00/p007965/p007965-2141-11-28-14-41,3476791_0010
  
index of last record processed successfully: 1976
--------------------------------------
processing record #: 1977, record_path_short: p007965-2141-11-30-07-00
  
index of last record processed successfully: 1977
--------------------------------------
processing record #: 1978, r

  1995,p00/p008057/p008057-2141-01-13-22-55,3526225_0045
  1995,p00/p008057/p008057-2141-01-13-22-55,3526225_0051
  1995,p00/p008057/p008057-2141-01-13-22-55,3526225_0054
  1995,p00/p008057/p008057-2141-01-13-22-55,3526225_0056
  1995,p00/p008057/p008057-2141-01-13-22-55,3526225_0058
  1995,p00/p008057/p008057-2141-01-13-22-55,3526225_0062
  1995,p00/p008057/p008057-2141-01-13-22-55,3526225_0063
  1995,p00/p008057/p008057-2141-01-13-22-55,3526225_0066
  1995,p00/p008057/p008057-2141-01-13-22-55,3526225_0067
  1995,p00/p008057/p008057-2141-01-13-22-55,3526225_0071
  1995,p00/p008057/p008057-2141-01-13-22-55,3526225_0073
  1995,p00/p008057/p008057-2141-01-13-22-55,3526225_0078
  1995,p00/p008057/p008057-2141-01-13-22-55,3526225_0080
  1995,p00/p008057/p008057-2141-01-13-22-55,3526225_0081
  1995,p00/p008057/p008057-2141-01-13-22-55,3526225_0084
  1995,p00/p008057/p008057-2141-01-13-22-55,3526225_0087
  1995,p00/p008057/p008057-2141-01-13-22-55,3526225_0088
  1995,p00/p008057/p008057-2141

In [ ]:
waveform_record_segment[segment_index][0]

In [ ]:
waveform_record_segment[segment_index][1]

In [ ]:
master_waveform_content_first_line

In [ ]:
master_waveform_content_first_line[2]

In [ ]:
int(master_waveform_content_first_line[2])